Sentence to Meters model by using
    - word-level embeddings using Bag-of-Words
        - character-level & pre-trained word vector need to be tried
    - Encoder-Decoder model
    - Encoder: RNN with GRU
    - Decoder: Attention-based RNN with GRU


In [1]:
# Requirements

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import csv

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0 # start of the sentence
EOS_token = 1 # end of the sentence


class Lang:  # helper class for word->index & index->word dictionaries
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addMeters(self, meters):
        for meter in meters:
            self.addWord(meter)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [4]:
def readLangs(filename):
    print("Reading lines...")
    
    xs = []
    ys = []
    with open('%s.csv' % filename, newline='') as myFile:  
        reader = csv.reader(myFile)
        for row in reader:
            xs.append(row[0])
            ys.append(row[1])

    # Split every line into pairs and normalize
    pairs = [[normalizeString(x), normalizeString(y)] for (x, y) in list(zip(xs, ys))]

    input_lang = Lang('sentence')
    output_lang = Lang('meters')

    return input_lang, output_lang, pairs

In [5]:
MAX_IN_LENGTH = 80
MAX_OUT_LENGTH = 15

def filterPair(p):
    return len(p[0].split(' ')) < MAX_IN_LENGTH and \
        len(p[1]) < MAX_OUT_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [22]:
def prepareData(filename):
    input_lang, output_lang, pairs = readLangs(filename)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
            input_lang.addSentence(pair[0])
            output_lang.addMeters(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words) # n_words will include 2 by SOS and EOS
    testsize = len(pairs)//10
    training_pairs = pairs[:testsize]
    test_pairs = pairs[testsize:]
    return input_lang, output_lang, training_pairs, test_pairs

input_lang, output_lang, pairs, test_pairs = prepareData('4B4V')
print("random example: ")
print(random.choice(pairs))
print("train pairs size: " + str(len(pairs)))
print("test pairs size: " + str(len(test_pairs)))

Reading lines...
Read 1151 sentence pairs
Trimmed to 1142 sentence pairs
Counting words...
Counted words:
sentence 2890
meters 4
random example: 
['i reade it in thy lookes, thy languisht grace,', '-+-+-+-+-+']
train pairs size: 114
test pairs size: 1028


In [7]:
# Encoder

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
# Simple Decoder

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
# Attention Decoder

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_in_length=MAX_IN_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_in_length = max_in_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_in_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
# Preparing Training Data

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def indexesFromMeters(lang, meters):
    return [lang.word2index[meter] for meter in meters]


def tensorFromSentence(lang, sentence):
    if lang.name=="meters":
        indexes = indexesFromMeters(lang, sentence)
    else:
        indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
# Training the Model

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_in_length=MAX_IN_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_in_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
# helper functions for printing time and progress

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def evaluate(encoder, decoder, sentence, max_in_length=MAX_IN_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_in_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_in_length, max_in_length)

        for di in range(max_in_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [14]:
def getAccuracy(encoder, decoder, n=300):
    correctNum = 0
    for i in range(n):
        pair = test_pairs[i]
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ''.join(output_words)
        output_sentence = re.sub('\<EOS>$', '', output_sentence)
        if output_sentence==pair[1]:
            correctNum += 1
    accracy = correctNum / n
    return accracy

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss() # negative log likelihood loss

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            accuracy = getAccuracy(encoder, decoder, n=300)
            print('%s (%d %d%%), NLLLoss: %.4f, Accuracy: %.2f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg, accuracy))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [16]:
# Plotting results

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [20]:
def evaluateRandomly(encoder, decoder, n=20):
    for i in range(n):
        pair = random.choice(test_pairs)
        print('>', pair[0]) # input
        print('=', pair[1]) # expected output
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ''.join(output_words)
        print('<', output_sentence) # output
        print('')

In [19]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, n_iters=75000, print_every=5000) # below error is because I stopped the training by key interrupt

3m 3s (- 42m 43s) (5000 6%), NLLLoss: 0.1493, Accuracy: 0.30
6m 2s (- 39m 14s) (10000 13%), NLLLoss: 0.0061, Accuracy: 0.29
9m 7s (- 36m 30s) (15000 20%), NLLLoss: 0.0004, Accuracy: 0.28
12m 8s (- 33m 23s) (20000 26%), NLLLoss: 0.0002, Accuracy: 0.28
15m 5s (- 30m 10s) (25000 33%), NLLLoss: 0.0001, Accuracy: 0.29


KeyboardInterrupt: 

In [21]:
evaluateRandomly(encoder1, attn_decoder1)

> and then there was a pile of wood for which
= -+-+-+-+-+
< -+-+-+-+-+<EOS>

> come back to me in dreams, that i may give
= -+-+-+-+-+
< -+-+-+-+<EOS>

> hands, do what you're bid:
= -+-+-+
< -+-+-+-+<EOS>

> for sidelong would she bend, and sing
= -+-+-+-+
< -+-+-+-+-+<EOS>

> that brave vition each way free,
= -+-+-+-+
< -+-+-+-+-+<EOS>

> and half-inch fissures in the slipper
= -+-+-+-+-+
< -+-+-+-+<EOS>

> it best awith night.  come, civil night,
= -+-+-+-+-+
< -+-+-+-+<EOS>

> onthou art aand when towards thee
= -+-+-+-+-+
< -+-+-+-+<EOS>

> saynow you are not as you were
= +--+--+--+--
< -+-+-+-+-+<EOS>

> come, night; come, romethou day in night;
= -+-+-+-+-+
< -+-+-+-+-+<EOS>

> with anguish moist and fever-dew,
= -+-+-+-+
< -+-+-+-+<EOS>

> and learn me how to lose a winning match,
= -+-+-+-+-+
< -+-+-+-+<EOS>

> bear with my weakness: old brain is trou
= -+-+-+-+-+-
< --+--+--+<EOS>

> and hermits are coned with their cells
= -+-+-+-+-+
< -+-+-+-+<EOS>

> and students with t